#### 우리가 Agent 10(고객 감정 기반 CRM 메시지 자동 생성)을 만들기 위해 필요한 ‘기초 감성 모델’을 만드는 용도

In [47]:
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
df = pd.read_csv("/Users/mac/Desktop/amore/STUDY-DATA/first_week/data_csv/테스트용.csv", encoding="latin-1", header=None)

#### 1: 감성레이블 (0=부정, 4금정, 2중립)
#### 2: id
#### 3: 업로드 날짜
#### 4: 정체 불명의 태그 (NO_QUERY로 동일 -> drop)
#### 5: user
#### 6: text

In [48]:
pd.DataFrame(df)

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


#### index 3 불필요하여 drop

In [49]:
df.columns = ["label", "tweet", "date","flag", "user", "text"]
df = df.drop(columns=["flag"])
df = df.reset_index()
print(df.head())

   index  label       tweet                          date             user  \
0      0      0  1467810369  Mon Apr 06 22:19:45 PDT 2009  _TheSpecialOne_   
1      1      0  1467810672  Mon Apr 06 22:19:49 PDT 2009    scotthamilton   
2      2      0  1467810917  Mon Apr 06 22:19:53 PDT 2009         mattycus   
3      3      0  1467811184  Mon Apr 06 22:19:57 PDT 2009          ElleCTF   
4      4      0  1467811193  Mon Apr 06 22:19:57 PDT 2009           Karoli   

                                                text  
0  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1  is upset that he can't update his Facebook by ...  
2  @Kenichan I dived many times for the ball. Man...  
3    my whole body feels itchy and like its on fire   
4  @nationwideclass no, it's not behaving at all....  


#### index 필요 없어서 column 삭제 

In [50]:
df = df.drop(columns=["index"])
pd.DataFrame(df)

,label,tweet,date,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


#### 회귀 분석과 EDA 분석용 df 나눔

In [51]:
df_raw = df.copy()
df_model = df_raw[["label", "text"]].copy()

#### 기본전처리와 토큰화 코드는 미리 실행함. 텍스트 전처리 함수 실행하여 필요 없는 부분 제거

In [52]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http/S+","",text)   #URL 제거
    text = re.sub(r"@\w+","", text)     #@username 제거
    text = re.sub(r"[^a-z\s]", "", text)    #알파벳, 공백 제외 모두 제외
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w in stop_words and len(w)>1]
    return tokens

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### label=0d의 모든 단어를 모아서 frequency 계산

In [53]:
from tqdm import tqdm
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
df_neg = df[df["label"] == 0].copy()
all_words = []
for txt in tqdm(df_neg["text"].tolist()):
    all_words.extend(clean_text(txt))
word_freq = Counter(all_words)

[nltk_data] Downloading package punkt to /Users/mac/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/mac/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
100%|██████████| 800000/800000 [00:17<00:00, 46112.84it/s]


#### 상위 15개 단어 출력

In [54]:
word_freq.most_common(15)

[('to', 312629),
 ('the', 256226),
 ('my', 189116),
 ('and', 151193),
 ('is', 127618),
 ('it', 119013),
 ('in', 114412),
 ('for', 98698),
 ('you', 94093),
 ('of', 92566),
 ('me', 90472),
 ('so', 86813),
 ('on', 83580),
 ('have', 82491),
 ('but', 81309)]

#### 전형적인 영어 stopwords가 많이 나와서 리스트 확장하여 제거

In [58]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

custom_stop = {"im","ive","dont","cant","u","ur","rt","amp"}
stop_words = set(stopwords.words("english")).union(custom_stop)

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", " ", text)      # 링크 제거
    text = re.sub(r"@\w+", " ", text)         # @유저 제거
    text = re.sub(r"[^a-z\s]", " ", text)     # 알파벳/공백 제외 제거

    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words and len(w) > 1]

    return tokens

In [59]:
clean_text("RT @someone LOL I'm soooo happy!!! http://test.com")

['lol', 'soooo', 'happy']